In [17]:
#all packages which will be used
import numpy as np
import pandas as pd
from scipy.io import loadmat
import time
import random

#plot
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from matplotlib.axes import Axes

#sklearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
#from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.svm import LinearSVC
#to plot more than 2 dimensions 
from mlxtend.plotting import plot_decision_regions

#scipy
from scipy.stats import chisquare
from scipy.stats import chi2
from scipy.spatial import distance

#good performance classifier
from xgboost import XGBClassifier


#math
from math import log

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [19]:
train_set=pd.read_csv("C:\\Users\\hasee\\workspace\\lab\\patRecDat\\forStudents\\medData\\train_set.csv")
X_val=pd.read_csv("C:\\Users\\hasee\\workspace\\lab\\patRecDat\\forStudents\\medData\\val_feature.csv")
y_val=pd.read_csv("C:\\Users\\hasee\\workspace\\lab\\patRecDat\\forStudents\\medData\\val_label.csv")
X_train=train_set.drop(["label"],axis=1)
y_train=train_set["label"]

In [3]:
#Nearest Mean algorithm
def nn_predict(X_val,X_train,y_train):
    y_pred=[]
    y_proba_1=[]
    y_proba_2=[]
    class_1=X_train[y_train==1]
    class_2=X_train[y_train==2]
    mean_1=np.array(class_1.mean()).reshape(1,-1)
    mean_2=np.array(class_2.mean()).reshape(1,-1)
    cov_1=np.matrix(class_1.cov())
    cov_2=np.matrix(class_2.cov())
    print("KNN start")
    start=time.time()
    dis_1=distance.cdist(X_val,mean_1,metric='mahalanobis',V=cov_1).ravel()
    dis_2=distance.cdist(X_val,mean_2,metric='mahalanobis',V=cov_2).ravel()
# get the k nearest mahalanobis distance
    for index in range(len(X_val)):
        if index%1000==0:
            print("the index is %d and time is %5.1f second"%(index,(time.time()-start)))
        if dis_1[index]<dis_2[index]:
                y_pred.append(np.float64(1.0)) #added the weight to the vote
        else:
                y_pred.append(np.float64(2.0))
        y_proba_1.append(dis_2[index]/(dis_1[index]+dis_2[index]))
        y_proba_2.append(dis_1[index]/(dis_1[index]+dis_2[index]))
    print("KNN STOP")
    y_pred=pd.Series(y_pred)
    return y_pred,y_proba_1,y_proba_2
    
        

In [12]:
def knn_predict(X_val,X_train,y_train,k_value,weight):
    assert weight in ['uniform','distance']
    if weight == 'distance':
        y_pred=[]
        y_proba_1=[]
        y_proba_2=[]
        cov_train=np.matrix(X_train.cov())
        print("KNN start")
        start=time.time()
        for test_index in range(len(X_val)):
            if test_index%100==0:
                print("the X_val is %d and time is %5.1f minute"%(test_index,(time.time()-start)/60))
            good=0
            bad=0
            sample=np.array(X_val.iloc[test_index]).reshape(1,-1)
            m_dis=pd.Series((distance.cdist(X_train,sample,metric='mahalanobis',V=cov_train)).ravel())\
            .sort_values(ascending=True)#average computing time 0.12s
            dis_set=m_dis[:k_value]# get the k nearest mahalanobis distance
            for index in dis_set.index:
                if y_train.loc[index] == 1.0:
                    good += 1/dis_set.loc[index] #added the weight to the vote
                else:
                    bad += 1/dis_set.loc[index]
            if good > bad :
                y_pred.append(np.float64(1.0))
            else:
                y_pred.append(np.float64(2.0))
            y_proba_1.append(good/(good+bad))
            y_proba_2.append(bad/(good+bad))
        print("KNN STOP")
        y_pred=pd.Series(y_pred)
    if weight == 'distance':
        y_pred=[]
        y_proba_1=[]
        y_proba_2=[]
        cov_train=np.matrix(X_train.cov())
        print("KNN start")
        start=time.time()
        for test_index in range(len(X_val)):
            if test_index%100==0:
                print("the X_val is %d and time is %5.1f minute"%(test_index,(time.time()-start)/60))
            good=0
            bad=0
            sample=np.array(X_val.iloc[test_index]).reshape(1,-1)
            m_dis=pd.Series((distance.cdist(X_train,sample,metric='mahalanobis',V=cov_train)).ravel())\
            .sort_values(ascending=True)#average computing time 0.12s
            dis_set=m_dis[:k_value]# get the k nearest mahalanobis distance
            for index in dis_set.index:
                if y_train.loc[index] == 1.0:
                    good += 1 #without weight
                else:
                    bad += 1
            if good > bad :
                y_pred.append(np.float64(1.0))
            else:
                y_pred.append(np.float64(2.0))
            y_proba_1.append(good/(good+bad))
            y_proba_2.append(bad/(good+bad))
        print("KNN STOP")
        y_pred=pd.Series(y_pred)
    return y_pred,y_proba_1,y_proba_2

In [13]:
MODE= 'SVM'  # random forest: 'RF', SVM: 'SVM', Nearest Neighbors: 'NN' , k nearest neighbours:'KNN', XGB: 'XGB'

In [26]:
BEST_SCORE=0
assert MODE in ['SVM', 'RF','XGB','KNN']
if MODE == 'KNN':
    for weight in ['distance','uniform']:
        for k_value in [3,4,5,6,7]:
            print("now the knn model is "+weight+" and the k_value is "+str(k_value))
            y_pred_knn,y_proba_1_knn,y_proba_2_knn=knn_predict(X_val,X_train,y_train,k_value,weight=weight)
            y_score=f1_score(y_val,y_pred)
            print("the knn model is "+weight+" and the k_value is "+str(k_value)+" has the score "+str(y_score))
            if y_score > BEST_SCORE:
                BEST_SCORE = y_score
                print("knn model "+weight+" and k_value "+str(k_value)+"is the best")
if MODE == 'SVM' :
    for C in [1,10,100,1000]:
        print("now the svm model C value is "+str(C))
        svm_clf=LinearSVC(loss='hinge',dual=True,C=C,random_state=42)
        #parameter 'dual' , due to the http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
        # dual = False better if n_samples>n_features
        svm_clf.fit(X_train,y_train)
        y_pred=svm_clf.predict(X_val)
        y_score=f1_score(y_val,y_pred)
        print("now the svm model C value is "+str(C)+" has the score "+str(y_score))
        if y_score > BEST_SCORE:
            BEST_SCORE = y_score
            print("now the svm model C value is "+str(C)+" is the best")
            
        
                
        
        

now the svm model C value is 1
now the svm model C value is 1 has the score 0.975893182075
now the svm model C value is 1is the best
now the svm model C value is 10
now the svm model C value is 10 has the score 0.975815208863
now the svm model C value is 100
now the svm model C value is 100 has the score 0.975846680562
now the svm model C value is 1000
now the svm model C value is 1000 has the score 0.952612408337
